# Load packages

In [1]:
# 'private_twitter_credentials' contains my Twitter credentials. Replace by 'twitter_credentials' with your credentials
import private_twitter_credentials
import twitter
import datetime
import pandas as pd
import time

TodaysDate = time.strftime("%Y-%m-%d-%H-%M")

# Seeting up twitter authentication

In [2]:
consumer_key = private_twitter_credentials.consumer_key
consumer_secret = private_twitter_credentials.consumer_secret
access_token = private_twitter_credentials.access_token
access_token_secret = private_twitter_credentials.access_token_secret

api = twitter.Api(
    consumer_key         =   consumer_key,
    consumer_secret      =   consumer_secret,
    access_token_key     =   access_token,
    access_token_secret  =   access_token_secret,
    tweet_mode = 'extended'
)

# Functions

Class `TweetMiner` class contains two methods: 

* `mine_user_tweets` which mine user's tweets making use of [Get user_timeline](https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline)

* `search_tweets` which mine tweets using [GetSearch](https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets)

`search_tweets` gives you possibility to perform queries. You can for instances perform a search at Twitter and copy what comes after `q` in your browser.

For example, if I search `@picnic @JumboSupermarkt @albertheijn covid-19` I have after `q` : `%40picnic%20%40JumboSupermarkt%20%40albertheijn%20covid-19&src=typed_query`. 


In [3]:
import datetime

class TweetMiner(object):
    """ Make possible obtaining tweets using twitter user id (mine_user_tweets) or performing a standard Twitter 
    API search"""

    
    def __init__(self, api, result_limit = 20, max_pages = 40):
        """result_limit = count that can take max 200 (mine_user_tweets) and max 100 (search_tweets)"""
        
        self.api = api        
        self.result_limit = result_limit
        self.max_pages = max_pages
        

    def mine_user_tweets(self, user, mine_retweets=False):
        """ Mine tweets of user = screen_name or user_id"""

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= self.max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1, 
                                                        include_rts=mine_retweets)
                statuses = [ _.AsDict() for _ in statuses]
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, include_rts=mine_retweets)
                statuses = [_.AsDict() for _ in statuses]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'mined_at':         datetime.datetime.now(),
                        'created_at':       item['created_at'],
                        'tweet_id':         item['id'],
                        'tweet_id_str':     item['id_str'],
                        'screen_name':      item['user']['screen_name'],
                        'favorite_count':   item['favorite_count'],
                        'text':             item['full_text'],
                        'source':           item['source'],
                        'language':         item['lang'],
                        'retweet_count':    item['retweet_count'],
                        'user_favourites_count': item['user']['favourites_count'],
                        'followers_count':  item['user']['followers_count'],
                        'friends_count':    item['user']['friends_count']
                    }
            
                
                except:
                    mined = {
                        'mined_at':         datetime.datetime.now(),
                        'created_at':       item['created_at'],
                        'tweet_id':         item['id'],
                        'tweet_id_str':     item['id_str'],
                        'screen_name':      item['user']['screen_name'],
                        'favorite_count':   item['favorite_count'],
                        'text':             item['full_text'],
                        'source':           item['source'],
                        'language':         item['lang'],
                        'retweet_count':    0,
                        'user_favourites_count': item['user']['favourites_count'],
                        'followers_count':  item['user']['followers_count'],
                        'friends_count':    item['user']['friends_count']
                        }
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
        return data
    
    def search_tweets(max_pages = 20, count = 20, raw_query = None, result_type = 'mixed'):
        """ Search tweets """

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses = api.GetSearch(raw_query=raw_query, count = count, result_type=result_type, max_id=last_tweet_id - 1)
                statuses = [ _.AsDict() for _ in statuses]
            else:
                statuses = api.GetSearch(raw_query=raw_query, count = count, result_type=result_type)
                statuses = [_.AsDict() for _ in statuses]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'mined_at':                datetime.datetime.now(),
                        'created_at':              item['created_at'],
                        'tweet_id':                item['id'],
                        'tweet_id_str':            item['id_str'],
                        'in_reply_to_screen_name': item['in_reply_to_screen_name'],
                        'in_reply_to_status_id':   item['in_reply_to_status_id'],
                        'in_reply_to_user_id':     item['in_reply_to_user_id'],
                        'language':                item['lang'],
                        'text':                    item['full_text'],
                        'hashtags':                item['hashtags'],
                        'source':                  item['source'],
                       # info about user
                        'screen_name':             item['user']['screen_name'],
                        'user_tweet_id':           item['user']['id'],
                        'user_tweet_id_str':       item['user']['id_str'],
                        'user_favourites_count':   item['user']['favourites_count'],
                        'followers_count':         item['user']['followers_count'],
                        'friends_count':           item['user']['friends_count']
                    }
                    
                except:
                    mined = {
                        'mined_at':                datetime.datetime.now(),
                        'created_at':              item['created_at'],
                        'tweet_id':                item['id'],
                        'tweet_id_str':            item['id_str'],
#                         'in_reply_to_screen_name': item['in_reply_to_screen_name'],
#                         'in_reply_to_status_id':   item['in_reply_to_status_id'],
#                         'in_reply_to_user_id':     item['in_reply_to_user_id'],
                        'language':                item['lang'],
                        'text':                    item['full_text'],
                        'hashtags':                item['hashtags'],
                        'source':                  item['source'],
                       # info about user
                        'screen_name':             item['user']['screen_name'],
                        'user_tweet_id':           item['user']['id'],
                        'user_tweet_id_str':       item['user']['id_str'],
                        'user_favourites_count':   item['user']['favourites_count'],
                        'followers_count':         item['user']['followers_count'],
                        'friends_count':           item['user']['friends_count']
                    }
                                            
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
        return data

In [4]:
def processing_and_saving(df, file_name, mine_user_twitter=1):
    """ Save retrieved tweets in csv file.
    
    Input:
    
    df : dataframe of tweets'data
    file_name: name with which the csv will be saved (without extension)
    mine_user_twitter: Indicates if df came contains tweets from a twitter user, i.e., was obtained using 
    GetUserTimeline since the information obtained from this method is different from an API search from GetSearch
    
    """
    
    TodaysDate = time.strftime("%Y-%m-%d-%H-%M")

    
    # Create columns 'year', 'month', 'day', 'hour', 'min' from 'created_at'
    df['created_at'] = pd.to_datetime(df['created_at'])
        
    df['year'] = df['created_at'].dt.year 
    df['month'] = df['created_at'].dt.month 
    df['day'] = df['created_at'].dt.day 
    df['hour'] = df['created_at'].dt.hour 
    df['minute'] = df['created_at'].dt.minute
    df['day_of_week'] = df['created_at'].dt.weekday
    
    if mine_user_twitter:
    
        df = df[['mined_at', 'screen_name', 'tweet_id', 'tweet_id_str', 'created_at', 'year', 'month', 'day','day_of_week', 
             'hour', 'minute', 'retweet_count', 'favorite_count', 'source', 'language', 'user_favourites_count', 
             'followers_count','friends_count','text']]
    else:
        df = df[['mined_at', 'tweet_id', 'tweet_id_str', 'in_reply_to_screen_name','in_reply_to_status_id',
                 'in_reply_to_user_id', 'hashtags','source','language', 'created_at', 'year', 'month', 'day','day_of_week', 
                 'hour', 'minute','screen_name','user_tweet_id','user_tweet_id_str','user_favourites_count','followers_count',
                 'friends_count', 'text']]
        
    df.sort_values(by='created_at',inplace = True)
    df.reset_index(drop = True, inplace = True)
    
    
    df.to_csv("../data/tweets/"+file_name+"_"+TodaysDate+".csv", index = False)
    
    return df
    
    

# Getting twitter by user

The goal of this project is to check the sentiment of users towards the main providers of online grocery shopping, i.e., Jumbo Supermarkten, AH, and Picnic.


## Instantiate the class
---

Make sure you pass the keys dictionary and the api as arguments.

**Check:** call the object's `mine_user_tweets()` method, providing a user to pull the tweets of.

In [5]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=20, max_pages = 200)

In [6]:
jairbolsonaro = miner.mine_user_tweets(user="jairbolsonaro")

In [7]:
for x in range(5):
    print(jairbolsonaro[x]['text'])
    print('--')

8. Recursos liberados para atender estados da Amazônia Legal, desde compra de máquinas até construções de vias para facilitação de locomoção da população;

9. A superintendência do Desenvolvimento do Centro-Oeste garantiu recursos para melhorar a infraestrutura de cidades goianas
--
6. Entregues 349 moradias em Goiás e Pará, focando na dignidade das pessoas mais humildes;

7. Em Minas Gerais, o @mdregional_br liberou recursos para ampliação da produção irrigada no Norte e Nordeste do estado;
--
4. O @mdregional_br também selecionou 6 projetos para melhorar e ampliar os sistemas de abastecimento de água e saneamento básico em São Paulo;

5. Foi autorizada a retomada de obras de 730 moradias em Suzano (SP) e Tabatinga (AM), gerando mais 2,6 mil empregos nos locais;
--
2. Em São Vicente-SP repasse de verbas para reconstrução da ponte A Tribuna;

3. Na cidade de São Bernardo do Campo-SP,  as obras de contenção de encostas e entregas de mais unidades habitacionais e andamento de mais ações 

In [8]:
df_bolsonaro = processing_and_saving(pd.DataFrame(jairbolsonaro), 'jairbolsonaro')

In [9]:
df_bolsonaro.head()

,mined_at,screen_name,tweet_id,tweet_id_str,created_at,year,month,day,day_of_week,hour,minute,retweet_count,favorite_count,source,language,user_favourites_count,followers_count,friends_count,text
0,2020-06-20 23:24:33.771297,jairbolsonaro,1116868726866042881,1116868726866042881,2019-04-13 01:00:29+00:00,2019,4,13,5,1,0,3215,27779,"<a href=""http://twitter.com/download/iphone"" r...",pt,2758,6667529,515,Nossa política é de mercado aberto e de não in...
1,2020-06-20 23:24:33.771297,jairbolsonaro,1116869473787625472,1116869473787625472,2019-04-13 01:03:27+00:00,2019,4,13,5,1,3,3114,24198,"<a href=""http://twitter.com/download/iphone"" r...",und,2758,6667529,515,https://t.co/olbvIPqajX
2,2020-06-20 23:24:33.771297,jairbolsonaro,1117021839123988482,1117021839123988482,2019-04-13 11:08:54+00:00,2019,4,13,5,11,8,3462,21712,"<a href=""http://twitter.com/download/iphone"" r...",pt,2758,6667529,515,"A Embrapa, em parceria com a Funai e outros ór..."
3,2020-06-20 23:24:33.771297,jairbolsonaro,1117070706712379392,1117070706712379392,2019-04-13 14:23:05+00:00,2019,4,13,5,14,23,3119,30028,"<a href=""http://twitter.com/download/iphone"" r...",und,2758,6667529,515,🇧🇷👍🏻👉🏻👉🏻 https://t.co/uvoY9JSG6I
4,2020-06-20 23:24:33.771297,jairbolsonaro,1117086242502598661,1117086242502598661,2019-04-13 15:24:49+00:00,2019,4,13,5,15,24,4972,30649,"<a href=""http://twitter.com/download/iphone"" r...",pt,2758,6667529,515,O Governo Federal trabalhando para concluir ro...


In [10]:
df_bolsonaro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2996 entries, 0 to 2995
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   mined_at               2996 non-null   datetime64[ns]     
 1   screen_name            2996 non-null   object             
 2   tweet_id               2996 non-null   int64              
 3   tweet_id_str           2996 non-null   object             
 4   created_at             2996 non-null   datetime64[ns, UTC]
 5   year                   2996 non-null   int64              
 6   month                  2996 non-null   int64              
 7   day                    2996 non-null   int64              
 8   day_of_week            2996 non-null   int64              
 9   hour                   2996 non-null   int64              
 10  minute                 2996 non-null   int64              
 11  retweet_count          2996 non-null   int64            

In [11]:
min(df_bolsonaro.created_at),max(df_bolsonaro.created_at)

(Timestamp('2019-04-13 01:00:29+0000', tz='UTC'),
 Timestamp('2020-06-20 19:46:37+0000', tz='UTC'))

# Applying GetSearch to search for a defined query

Making a search in Twitter using `@jairbolsonaro covid-19 corona` gave us `q=%40jairbolsonaro%20covid-19%20corona&src=typed_query`

In [12]:
raw_query="q=%40jairbolsonaro%20covid-19%20corona&src=typed_query"
search_bolsonaro_covid = TweetMiner.search_tweets(max_pages = 15, count = 20, raw_query = raw_query, result_type = 'mixed')

In [13]:
len(search_bolsonaro_covid)

225

In [14]:
search_bolsonaro_covid[0]

{'mined_at': datetime.datetime(2020, 6, 20, 23, 28, 8, 65310),
 'created_at': 'Sat Jun 20 20:26:50 +0000 2020',
 'tweet_id': 1274438583731064835,
 'tweet_id_str': '1274438583731064835',
 'in_reply_to_screen_name': 'secomvc',
 'in_reply_to_status_id': 1274416731453259779,
 'in_reply_to_user_id': 1158389772920020993,
 'language': 'pt',
 'text': '@secomvc @jairbolsonaro @minsaude @funaioficial @DefesaGovBr @anvisa_oficial @MinEconomia É urgente q o Ministério da Saúde cumpra sua função de liderar a crise do Corona\nEle tem q FALAR c/ a população, unir estados e municípios p/ q juntos evitem a disseminação\nEsse 👇 é um triste exemplo de q as pessoas NÃO podem ficar doentes AO MESMO TEMPO\n\nhttps://t.co/TCaK16LAEB',
 'hashtags': [],
 'source': '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>',
 'screen_name': 'SandraMTRibeir1',
 'user_tweet_id': 1101575981356236803,
 'user_tweet_id_str': '1101575981356236803',
 'user_favourites_count': 1455,
 'followers_co

In [15]:
df_search_bolsonaro_corona = processing_and_saving(pd.DataFrame(search_bolsonaro_covid),"search_bolsonaro_corona",mine_user_twitter=0)

In [16]:
df_search_bolsonaro_corona.tail()

,mined_at,tweet_id,tweet_id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,hashtags,source,language,created_at,...,day_of_week,hour,minute,screen_name,user_tweet_id,user_tweet_id_str,user_favourites_count,followers_count,friends_count,text
220,2020-06-20 23:28:08.696027,1274438583731064835,1274438583731064835,secomvc,1.274417e+18,1.158390e+18,[],"<a href=""http://twitter.com/#!/download/ipad"" ...",pt,2020-06-20 20:26:50+00:00,...,5,20,26,SandraMTRibeir1,1101575981356236803,1101575981356236803,1455,20,342,@secomvc @jairbolsonaro @minsaude @funaioficia...
221,2020-06-20 23:28:11.781770,1274438583731064835,1274438583731064835,secomvc,1.274417e+18,1.158390e+18,[],"<a href=""http://twitter.com/#!/download/ipad"" ...",pt,2020-06-20 20:26:50+00:00,...,5,20,26,SandraMTRibeir1,1101575981356236803,1101575981356236803,1455,20,342,@secomvc @jairbolsonaro @minsaude @funaioficia...
222,2020-06-20 23:28:09.875278,1274438583731064835,1274438583731064835,secomvc,1.274417e+18,1.158390e+18,[],"<a href=""http://twitter.com/#!/download/ipad"" ...",pt,2020-06-20 20:26:50+00:00,...,5,20,26,SandraMTRibeir1,1101575981356236803,1101575981356236803,1455,20,342,@secomvc @jairbolsonaro @minsaude @funaioficia...
223,2020-06-20 23:28:11.514203,1274438583731064835,1274438583731064835,secomvc,1.274417e+18,1.158390e+18,[],"<a href=""http://twitter.com/#!/download/ipad"" ...",pt,2020-06-20 20:26:50+00:00,...,5,20,26,SandraMTRibeir1,1101575981356236803,1101575981356236803,1455,20,342,@secomvc @jairbolsonaro @minsaude @funaioficia...
224,2020-06-20 23:28:08.065310,1274438583731064835,1274438583731064835,secomvc,1.274417e+18,1.158390e+18,[],"<a href=""http://twitter.com/#!/download/ipad"" ...",pt,2020-06-20 20:26:50+00:00,...,5,20,26,SandraMTRibeir1,1101575981356236803,1101575981356236803,1455,20,342,@secomvc @jairbolsonaro @minsaude @funaioficia...


In [17]:
df_search_bolsonaro_corona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   mined_at                 225 non-null    datetime64[ns]     
 1   tweet_id                 225 non-null    int64              
 2   tweet_id_str             225 non-null    object             
 3   in_reply_to_screen_name  90 non-null     object             
 4   in_reply_to_status_id    90 non-null     float64            
 5   in_reply_to_user_id      90 non-null     float64            
 6   hashtags                 225 non-null    object             
 7   source                   225 non-null    object             
 8   language                 225 non-null    object             
 9   created_at               225 non-null    datetime64[ns, UTC]
 10  year                     225 non-null    int64              
 11  month                    225 non

REFS:

> https://python-twitter.readthedocs.io/en/latest/index.html

> https://developer.twitter.com/en/docs

Info:

1st corona case in The Netherlands: 27/02/2020

1st corona case in Brazil: 26/02/2020


**TODO**

- Check functions to see if I have all I need.
- Generalize as much as possible
- Order infor per date (created_at?)
- Should I have a function also for search?
- Retrieve info for:
    - jairbolsonaro
    - picnic
    - JumboSupermarkt
    - albertheijn
    - evanescence
    - WTofficial
    
- Include in class `TweetMiner` a function to user `GetSearch`

Using [`python-twitter`](https://python-twitter.readthedocs.io/en/latest/getting_started.html) since I've been facing some time out problems with [`Tweepy`](http://docs.tweepy.org/en/latest/getting_started.html).

In [ ]:
# !pip install python-twitter

# Load packages

# Setting up twitter authentication

# Functions

Function `mine_user_tweets` made use of [Get user_timeline](https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline)


In [ ]:
import datetime
#TweetMiner function from Mike Roman

# some modification added by me - 15/04/19

class TweetMiner(object):

    
    def __init__(self, api, result_limit = 20, max_pages = 40):
        """result_limit = count that can take max 200"""
        
        self.api = api        
        self.result_limit = result_limit
        self.max_pages = max_pages
        

    def mine_user_tweets(self, user, mine_retweets=False):
        """ Mine tweets of user = screen_name or user_id"""

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= self.max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1, 
                                                        include_rts=mine_retweets)
                statuses = [ _.AsDict() for _ in statuses]
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, include_rts=mine_retweets)
                statuses = [_.AsDict() for _ in statuses]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   item['retweet_count'],
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                        'favorite_count':  item['favorite_count'],
                        'tweet_id_str':    item['id_str'],
                        'source':          item['source'],
                        
                    }
                    
            
                
                except:
                        mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   0,
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
#                         'favorite_count':  item['favorite_count'],
                        'tweet_id_str':    item['id_str'],
                        'source':          item['source'],
                    }
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
        return data

In [ ]:
def processing_and_saving(df, file_name):
    
    TodaysDate = time.strftime("%Y-%m-%d-%H-%M")

    
    # Create columns 'year', 'month', 'day', 'hour', 'min' from 'created_at'
    df['created_at'] = pd.to_datetime(df['created_at'])
        
    df['year'] = df['created_at'].dt.year 
    df['month'] = df['created_at'].dt.month 
    df['day'] = df['created_at'].dt.day 
    df['hour'] = df['created_at'].dt.hour 
    df['minute'] = df['created_at'].dt.minute
    df['day_of_week'] = df['created_at'].dt.weekday
    
    df.sort_values(by='created_at',inplace = True)
    
    df = df[['mined_at', 'handle','tweet_id', 'tweet_id_str','created_at', 
             'year', 'month', 'day','day_of_week', 'hour', 'minute','retweet_count', 'source', 'text']]
    
    df.to_csv("./data_tweets/"+file_name+"_"+TodaysDate+".csv", index = False)
    
    return df
    
    

# Getting twitter by user

## Instantiate the class
---

Make sure you pass the keys dictionary and the api as arguments.

**Check:** call the object's `mine_user_tweets()` method, providing a user to pull the tweets of.

In [ ]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=20, max_pages = 200)

In [ ]:
jairbolsonaro = miner.mine_user_tweets(user="jairbolsonaro")

In [ ]:
for x in range(5):
    print(jairbolsonaro[x]['text'])
    print('--')

## Convert the tweet ouputs to a pandas DataFrame
### jairbolsonaro

In [ ]:
jairbolsonaro

In [ ]:
df_bolsonaro = processing_and_saving(pd.DataFrame(jairbolsonaro), 'jairbolsonaro')

In [ ]:
df_bolsonaro.head()

In [ ]:
df_bolsonaro.info()

In [ ]:
min(df_bolsonaro.created_at),max(df_bolsonaro.created_at)

In [ ]:
df_bolsonaro.sort_values(by='created_at')

### picnic

In [ ]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=20, max_pages = 100)
picnic = miner.mine_user_tweets(user="picnic")
df_picnic = processing_and_saving(pd.DataFrame(picnic), "picnic")


In [ ]:
df_picnic.info()

In [ ]:
min(df_picnic.created_at),max(df_picnic.created_at)

### JumboSupermarkt

In [ ]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=20, max_pages = 200)
JumboSupermarkt = miner.mine_user_tweets(user="JumboSupermarkt")
df_JumboSupermarkt = processing_and_saving(pd.DataFrame(JumboSupermarkt), "JumboSupermarkt")


In [ ]:
df_JumboSupermarkt.info()

In [ ]:
min(df_JumboSupermarkt.created_at),max(df_JumboSupermarkt.created_at)

### albertheijn

In [ ]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=20, max_pages = 300)
albertheijn = miner.mine_user_tweets(user="albertheijn")
df_albertheijn = processing_and_saving(pd.DataFrame(albertheijn), "albertheijn")


In [ ]:
df_albertheijn.info()

In [ ]:
min(df_albertheijn.created_at),max(df_albertheijn.created_at)

### evanescence

In [ ]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=20, max_pages = 300)
evanescence = miner.mine_user_tweets(user="evanescence")
df_evanescence = processing_and_saving(pd.DataFrame(evanescence), "evanescence")


In [ ]:
df_evanescence.info()

In [ ]:
min(df_evanescence.created_at),max(df_evanescence.created_at)

In [ ]:
df_evanescence.tail()

### Within Temptation - WTofficial

In [ ]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=20, max_pages = 300)
WTofficial = miner.mine_user_tweets(user="WTofficial")
df_WTofficial = processing_and_saving(pd.DataFrame(WTofficial), "WTofficial")


In [ ]:
df_WTofficial.info()

In [ ]:
min(df_WTofficial.created_at),max(df_WTofficial.created_at)

# Experimenting with GetSearch

In [ ]:
def search_tweets(max_pages = 20, count = 20, raw_query = None, result_type = 'mixed'):
        """ Search tweets """

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses = api.GetSearch(raw_query=raw_query, count = count, result_type=result_type, max_id=last_tweet_id - 1)
                statuses = [ _.AsDict() for _ in statuses]
            else:
                statuses = api.GetSearch(raw_query=raw_query, count = count, result_type=result_type)
                statuses = [_.AsDict() for _ in statuses]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   item['retweet_count'],
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                        'favorite_count':  item['favorite_count'],
                        'tweet_id_str':    item['id_str'],
                        'source':          item['source'],
                        
                    }
                    
            
                
                except:
                        mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   0,
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
#                         'favorite_count':  item['favorite_count'],
                        'tweet_id_str':    item['id_str'],
                        'source':          item['source'],
                    }
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
        return data

In [ ]:
raw_query="q=jairbolsonaro%2C%20corona%2C%20covid&src=typed_query"
results = search_tweets(max_pages = 15, count = 20, raw_query = raw_query, result_type = 'mixed')

In [ ]:
len(results)

In [ ]:
results[0]

In [ ]:
df_search_bolsonaro_corona = processing_and_saving(pd.DataFrame(results),"search_bolsonaro_corona")

In [ ]:
df_search_bolsonaro_corona.tail()

In [ ]:
df_search_bolsonaro_corona.info()

In [ ]:
results = api.GetSearch(raw_query="q=picnic", count = 100, result_type='popular')

In [ ]:
results

In [ ]:
raw_query = "q=picnic%20jumbo%20ah%20delivery&src=typed_query"

picnic_ah_jumbo_search = search_tweets(max_pages = 1, count = 5, raw_query = raw_query, result_type = 'mixed')
picnic_ah_jumbo_search

In [ ]:
df_picnic_ah_jumbo_search = processing_and_saving(pd.DataFrame(picnic_ah_jumbo_search),"picnic_ah_jumbo_search")

In [ ]:
df_picnic_ah_jumbo_search